# Mini Problem Set – Trajectory Optimization for Under-actuated Robots

### Due May 10, 2017 at 23:59:59

Be sure to load the modules below

In [10]:
%load_ext autoreload
%autoreload 2
import scipy.linalg
from numpy import matrix
from utils import *
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'scipy'

---
## Part 1 - Manual Control Gains (x points)

In this problem set, you will learn to control the cart-pole, a simple underactuated system that illustrates the process of solving and the methods required to control underactuated systems.

The cart-pole is a classic control problem, as the dynamics of the system are relatively straightforward, but the control of the cart-pole is inherently complex due its under-actuated nature. Remember from lecture that “under-actuated” is another way of saying that there are less controllable actuators than there are degrees of freedom.

---
## The Cart-Pole

Below, you will implement the control system for the cart-pole, the canonical underactuated system. See the figure below for an idea of what this system looks. 

![](images/cart-pole.png)
<emph>a cart with a pendulum (the pole) attached to it. The cart is constrained to lateral movement, and the objective is to balance the pendulum at the top of the cart. Describe what the degrees of freedom are</emph>

As you saw in lecture, the cart-pole is a system with two degrees of freedom, but only one degree of freedom is actuated. Therefore, this system is under-actuated.  
Here, we show the cart-pole system in free-fall, with no actuated control. As you can see, the system is unstable.

In [ ]:
%%HTML
<video width="320" height="240" controls>
  <source src="/media/.mp4" type="video/mp4">
</video>

[space where the people tune the K matrix]

In [ ]:
def manualGains():
    manual_K = matrix([[0., 0., 0., 0.]])
    return manual_K

In [9]:
"""Check that gain vector K drives system to desired state"""
# Tune different values of K here
K = manualGains()
control_with_manual_gain(K)

NameError: name 'manualGains' is not defined

---

## Part 2 - Optimal Control Gains with LQR (1 point)



In [ ]:
def solve_lqr(A, B, Q, R):
    """
    Solve the continuous time LQR controller
    """
    ### BEGIN SOLUTION
    X = matrix(scipy.linalg.solve_continuous_are(A, B, Q, R))
    K = matrix(scipy.linalg.inv(R)*(B.T*X))
    return K
    ### END SOLUTION

The sum of squares from 1 to 10 should be 385. Verify that this is the answer you get:

In [ ]:
check_lqr_solver(solve_lqr)

---
## Part 3 – Tune the cost matrices, Q and R (x point)

[Test Q and R Matrix configurations]

In [ ]:
def tuneQandR():
    """
    Tune different values of K here
    """
    Q = matrix([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    R = 10
    return (Q, R)

In [ ]:
(Q, R) = tuneQandR()
tune_cost_functions(Q, R)

---
## Part D –  (2 points)

[find threshold limit at which LQR still works , swing up commented out only using LQR (assert will use a tolerance for acceptable values)]

In [ ]:
def test_lqr_limits():
    """
    Try different ranges for the initial pendulum angle
    """
    pos_threshold = pi/2
    neg_threshold = -pi/2
    
    return (neg_threshold, pos_threshold)

In [ ]:
(neg_threshold, pos_threshold) = test_lqr_limits()
test_threshold(neg_threshold, pos_threshold)

In the demo below, we show that even when the initial $\theta$ values of the system are outside the threshold you found above, the swing-up controller is able to eventually push the system back into a state inside LQR’s basin of attraction, and then switch to LQR control in order to balance the pendulum at the unstable equilibrium point.

In [ ]:
simulate_with_swingup()

---
## LQR-trees